In [1]:
import pandas as pd
from custom_func import *

In [2]:
df_P02_006 = pd.read_excel(f"{INPUTS_PATH}/P2/P02_006.xlsx")
df_P02_007 = pd.read_excel(f"{INPUTS_PATH}/P2/P02_007.xlsx")

In [3]:
colnames = {
    "Область": "region",
    "Податки на одну особу (p2_01)":'p2_01_raw',
    "Дохід без міжбюдж. трансфертів (p2_02)":'p2_02_raw',
    "Дохід без міжбюдж. трансфертів порівняно з минулим періодом (p2_03)": "p2_03_raw",
    "Капітальні видатки (p2_04)":'p2_04_raw',
    "Плата за землю (p2_05)":'p2_05_raw'
}

df_P02_006 = pd.read_excel(f"{INPUTS_PATH}/P2/P02_006.xlsx").rename(columns=colnames)

In [4]:
normalize_parameter(df_P02_006, "p2_01_raw", "p2_01")
normalize_parameter(df_P02_006, "p2_02_raw", "p2_02")
normalize_parameter(df_P02_006, "p2_03_raw", "p2_03")
normalize_parameter(df_P02_006, "p2_04_raw", "p2_04")
normalize_parameter(df_P02_006, "p2_05_raw", "p2_05")

df_P02_006_fin = df_P02_006.loc[:, df_P02_006.columns.str.contains("region|^p")]

Емпіричні границі: (273.1319492833434, 1419.510256667407)
Коефіцієнти нормалізації: (0.0008723123889895594, -0.23825638318872844)
Параметр p2_01 нормалізовано і додано до таблиці

Емпіричні границі: (689342314.3199999, 6955395883.62)
Коефіцієнти нормалізації: (1.595900815306488e-10, -0.11001219614485493)
Параметр p2_02 нормалізовано і додано до таблиці

Емпіричні границі: (0.624658426803403, 1.168221845737069)
Коефіцієнти нормалізації: (1.8397117340268172, -1.1491914375489523)
Параметр p2_03 нормалізовано і додано до таблиці

Емпіричні границі: (64811628, 1646013700)
Коефіцієнти нормалізації: (6.324302362791237e-10, -0.04098883320967467)
Параметр p2_04 нормалізовано і додано до таблиці

Емпіричні границі: (60392282.4, 729124352.5)
Коефіцієнти нормалізації: (1.4953671951914363e-09, -0.09030863794369715)
Параметр p2_05 нормалізовано і додано до таблиці



---

# Розрахунки на даних, отриманих з BizTenders

In [5]:
df_P02_007_fin = df_P02_007.loc[df_P02_007["status"].eq("complete")].copy()

In [6]:
df_P02_007_fin["Чи відкриті торги?"] = df_P02_007_fin["procurementMethod"].eq("open").astype(int)
open_procur = df_P02_007_fin.loc[df_P02_007_fin['Чи відкриті торги?']==1].groupby('region')["tenderValue"].sum().rename('open')
all_procur = df_P02_007_fin.groupby('region')["tenderValue"].sum().rename('all')
df_P02_007_fin = pd.concat([open_procur, all_procur], axis=1).reset_index()
df_P02_007_fin["p2_06_raw"] = df_P02_007_fin["open"] / df_P02_007_fin["all"]

In [7]:
df_P02_007_fin_biz = df_P02_007_fin.copy()

---

# Розрахунки на даних bi.prozorro

In [8]:
df = pd.read_excel("./../v0.5/inputs/P2/P02_007_prozorro.xlsx")

open_procedures = ["Відкриті торги", "Відкриті торги із публікацією англ. мовою"]
df["Чи відкриті торги?"] = df["Процедура закупівлі"].isin(open_procedures).astype(int)

df = df.loc[df["Статуси процедур"].eq("Завершена закупівля")].copy()
open_procur = df.loc[df['Чи відкриті торги?']==1].groupby('region')["Очікувана вартість"].sum().rename('open')
all_procur = df.groupby('region')["Очікувана вартість"].sum().rename('all')
df_P02_007_fin = pd.concat([open_procur, all_procur], axis=1).reset_index()
df_P02_007_fin["p2_06_raw"] = df_P02_007_fin["open"] / df_P02_007_fin["all"]

# normalize_parameter(df_P02_007_fin, "p2_06_raw", "p2_06", min_bound=0)

In [9]:
df_P02_007_fin_bipro = df_P02_007_fin.copy()

---

In [10]:
pd.merge(
    df_P02_007_fin_bipro.rename(columns={"p2_06_raw": "p2_06_raw_biprozorro"}).drop(["open", "all"], 1),
    df_P02_007_fin_biz.rename(columns={"p2_06_raw": "p2_06_raw_bizTenders"}).drop(["open", "all"], 1),
    on="region"
)

,region,p2_06_raw_biprozorro,p2_06_raw_bizTenders
0,Івано-Франківська,0.585842,0.519161
1,Волинська,0.475664,0.461764
2,Вінницька,0.821416,0.873118
3,Дніпропетровська,0.473084,0.548829
4,Донецька,0.747781,0.779114
5,Житомирська,0.854600,0.877019
6,Закарпатська,0.734048,0.619623
7,Запорізька,0.673762,0.731062
8,Київська,0.663947,0.571182
9,Кіровоградська,0.721985,0.688914
